# Chapter 8. Load balancing

# Céline Comte

This Notebook computes numerical results of Chapter 8 in the Ph.D. thesis *Resource management in computer clusters: algorithm design and performance analysis* by Céline Comte. Specifically, the following piece of code computes the performance of two static load-balancing policies, referred to as *best static* and *uniform static*. A static load-balancing policy assigns each incoming job to a computer chosen at random, independently of the current cluster state, according to pre-determined routing probabilities. For each $i = 1,\ldots,I$, we let $p_i$ denote the probability that an incoming job is assigned to computer $i$. The exogeneous arrival rate is denoted by $\nu$.

## Package imports and global variable definitions

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# uncomment this line if you prefer dynamic matplotlib plots
# %matplotlib notebook

# change the default figure size
pylab.rcParams['figure.figsize'] = (10.0, 6.0)
pylab.rcParams['legend.fontsize'] = 12

In [3]:
# manipulate dataframes
import pandas as pd

In [4]:
# global variables
ρρ = append(linspace(5., .01, 400, endpoint = False), linspace(.01, 0, 10, endpoint = False))
nb_tokens = 6

## Performance of the static load-balancing policies

### Computations

**Independence.**
In steady state, the number of jobs at each computer is independent of the number of jobs at the other computers. Furthermore, for each $i = 1,\ldots,I$, the number of available tokens of computer $i$ is equal to the number of customers in an M/M/1/$\ell_i$ queue with load $\rho_i = \mu_i / (\nu p_i)$. Therefore, the stationary distribution of the Markov process defined by the macrostate $y = (y_1, \ldots, y_I)$ that counts the number of available tokens of each computer is given by
$$
\pi(y) = \prod_{i=1}^I \frac{ {\rho_i}^{y_i} }{ 1 + \rho_i + \ldots + {\rho_i}^{\ell_i} },
\quad \forall y \le \ell,
$$
Recall that we have
$$
1 + \rho_i + \ldots + {\rho_i}^{\ell_i} = 
\begin{cases}
    \frac{1 - {\rho_i}^{\ell_i + 1}}{1 - \rho_i}
    &\text{if $\rho_i \neq 1$}, \\
    \ell_i + 1
    &\text{if $\rho_i = 1$}.
\end{cases}
$$

**Loss probability.**
With this observation in mind, we can write the average loss probability $\beta$ as follows:
$$
\beta
= p_1 \beta_{|1} + p_2 \beta_{|2} + \ldots + p_I \beta_{|I},
$$
where $\beta_{|i}$ is the loss probability of jobs assigned to computer $i$.
The arrival process to this computer is Poisson with rate $\lambda_i = \nu p_i$.
Hence, we can apply PASTA property, which states that
$\beta_{|i}$ is also the probability that all tokens of computer $i$ are held by jobs in service,
that is, $y_i = 0$:
$$
\beta_{|i} = \frac1{ 1 + \rho_i + \ldots + {\rho_i}^{\ell_i} }.
$$

**Expected number of jobs.**
By linearity of the expectation, the expected number $L$ of jobs in the cluster is given by
$$
L = L_1 + \ldots + L_I,
$$
where, for each $i = 1,\ldots,I$,
$L_i$ is the expected number of jobs at computer $i$.
Since the computer states are independent, we have
$$
L_i = \frac
{ 0 + \rho_i + 2 {\rho_i}^2 + \ldots + \ell_i {\rho_i}^{\ell_i} }
{ 1 + \rho_i + {\rho_i}^2 + \ldots + {\rho_i}^{\ell_i} },
\quad \forall i = 1,\ldots,N.
$$

### Functions

The function ``mm1ℓ`` computes performance in an M/M/1/$\ell$ queue subject to the load $\rho$.

In [5]:
def mm1ℓ(ℓ, ρ):
    π = power(ρ, arange(ℓ+1))
    π /= sum(π)
    return π[0], ℓ - inner(π, arange(ℓ+1))

The function ``static`` computes performance under a given static load-balancing policy, defined by the routing probabilities given by the vector $p$.

In [6]:
def static(I, ℓ, μ, ν, p):
    β = zeros(I); L = zeros(I)
    result = []
    
    for i in range(I):
        β[i], L[i] = mm1ℓ(ℓ[i], μ[i] / (ν * p[i]))
        result += (β[i], L[i])
    result += [inner(p, β), sum(L)]
    
    return result

The function ``create_static_df`` calls the function ``static`` and creates a dataframe (from ``pandas`` library) that stores the results:

In [7]:
def create_static_df(I, ℓ, μ, ρρ, p, prefix = ""):
    data = [static(I, ℓ, μ, ρ * sum(μ), p) for ρ in ρρ]
    
    df = pd.DataFrame({'rho': ρρ})
    
    for i in range(I):
        df[prefix + 'betai' + str(i+1)] = [d[2*i] for d in data]
        df[prefix + 'etai' + str(i+1)] = (sum(μ) * p[i] / μ[i]) * ρρ * [1. - d[2*i] for d in data]
        df[prefix + 'Li' + str(i+1)] = [d[2*i+1] for d in data]
        df[prefix + 'gammai' + str(i+1)] = ρρ * sum(μ) * p[i] * [(1. - d[2*i]) / d[2*i+1] for d in data]

    df[prefix + 'beta'] = [d[2*I] for d in data]
    df[prefix + 'eta'] = ρρ * [1. - d[2*I] for d in data]
    df[prefix + 'L'] = [d[2*I+1] for d in data]
    df[prefix + 'gamma'] = ρρ * sum(μ) * [(1. - d[2*I]) / d[2*I+1] for d in data]
    
    return df

## A single job type

We consider the first scenario, described in Subsection 8.3.1.
There are $I = 10$ computers, each with $\ell = 6$ tokens.
The first half have a unit service capacity $\mu$
and the other half have a service capacity $4 \mu$.
There is a single job type, i.e., all jobs can be assigned to any computer.
The external arrival rate is denoted by $\nu$.

In [8]:
# parameters
I = 10
μ = ones(I); μ[I//2:] = 4.
ℓ = nb_tokens * ones(I, dtype = int)

# best static policy
best = ones(I)
best[I//2:] = 4.
best /= sum(best)

# uniform static policy
uni = ones(I)
uni /= sum(uni)

In [9]:
# compute the analytical results
best_static_df = create_static_df(I, ℓ, μ, ρρ, best)
uni_static_df = create_static_df(I, ℓ, μ, ρρ, uni)

In [10]:
# save them in csv
best_static_df.to_csv("data/single-best-static-exact.csv", index = False)
uni_static_df.to_csv("data/single-uni-static-exact.csv", index = False)

## Two job types

We consider the second scenario, described in Subsection 8.3.2.
There are $I = 10$ servers, each with $\ell = 6$ tokens.
All computer have the same unit capacity $\mu$.
There are two job types.
The jobs of the first type arrive at a unit rate $\nu$
and can be assigned to any of the first seven servers.
The jobs of the second type arrive at rate $4 \nu$
and can be assigned to any of the last seven servers.

In [11]:
# parameters
I = 10
μ = ones(I)
ℓ = nb_tokens * ones(I, dtype = int)

# best static policy
best = asarray([7, 7, 7, 12, 12, 12, 12, 12, 12, 12], dtype = float)
best /= sum(best)

# uniform static policy
uni = asarray([1, 1, 1, 5, 5, 5, 5, 4, 4, 4], dtype = float)
uni /= sum(uni)

In [12]:
# compute the analytical results
best_static_df = create_static_df(I, ℓ, μ, ρρ, best)
uni_static_df = create_static_df(I, ℓ, μ, ρρ, uni)

In [13]:
# save the results in csv
best_static_df.to_csv("data/multi-best-static-exact.csv", index = False)
uni_static_df.to_csv("data/multi-uni-static-exact.csv", index = False)